In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
ux_data = pd.read_csv('LTFS_UX_Score_test.csv')
ux_data.drop(['quit','asi','creation_date','nonfatale','device_id','custom_user_id','type_id','apierrorcode','name','name.1'],axis=1,inplace=True)
ux_data.head()

,usid,screen_duration,rage_count,apiduration,apistatus,crs,errorcount,app_launch_type,app_launch_time,user_defiened_event_tolerance,user_defiened_event_frustrated,case
0,1193007845,30400.0,NaN,1.303,s,0,NaN,COLD,949.0,20162019.0,NaN,Tolerance
1,1193007859,14000.0,NaN,0.047,s,0,NaN,COLD,668.0,NaN,NaN,sucess
2,1193007908,44400.0,NaN,1.521,s,0,NaN,COLD,1023.0,NaN,NaN,Tolerance
3,1193007933,59200.0,NaN,1.698,s,0,NaN,COLD,34913.0,NaN,NaN,fustrated
4,1193007956,25600.0,NaN,0.025,s,0,NaN,COLD,292.0,20337338.0,NaN,Tolerance


In [3]:
#Removing Null Values
ux_data['user_defiened_event_tolerance'] = ux_data['user_defiened_event_tolerance'].fillna(0)
ux_data['user_defiened_event_frustrated'] = ux_data['user_defiened_event_frustrated'].fillna(0)  
ux_data['rage_count'] = ux_data['rage_count'].fillna(0)
ux_data['apiduration'] = ux_data['apiduration'].fillna(0)
ux_data['apistatus'] = ux_data['apistatus'].fillna(0)
ux_data['errorcount'] = ux_data['errorcount'].fillna(0)
ux_data['app_launch_time'] = ux_data['app_launch_time'].fillna(0)

In [4]:
df = ux_data.copy()
df.head()

,usid,screen_duration,rage_count,apiduration,apistatus,crs,errorcount,app_launch_type,app_launch_time,user_defiened_event_tolerance,user_defiened_event_frustrated,case
0,1193007845,30400.0,0.0,1.303,s,0,0.0,COLD,949.0,20162019.0,0.0,Tolerance
1,1193007859,14000.0,0.0,0.047,s,0,0.0,COLD,668.0,0.0,0.0,sucess
2,1193007908,44400.0,0.0,1.521,s,0,0.0,COLD,1023.0,0.0,0.0,Tolerance
3,1193007933,59200.0,0.0,1.698,s,0,0.0,COLD,34913.0,0.0,0.0,fustrated
4,1193007956,25600.0,0.0,0.025,s,0,0.0,COLD,292.0,20337338.0,0.0,Tolerance


In [31]:
#weights 
rt = 2
ad = 1.5
ca = 5000
lt = 3
eve = 2.5

df['s'] = 0
df['t'] = 0
df['f'] = 0

for i,r in df.iterrows():
    summ = 0
    sf = 0
    st = 0
    
    #conditions frustrated
    if r['rage_count'] >= 4 :
        sf += ( r['rage_count'] * rt )
    if r['apiduration'] >= 6:
        sf += (r['apiduration'] * ad )
    if r['apistatus'] == 'f':
        sf += ca
    if r['crs'] in (1,2):
        sf += ca
    if r['errorcount'] >=4 : 
        sf += 1
    if r['user_defiened_event_frustrated'] != 0 :
        sf += (1 * eve )
    if r["app_launch_type"] == "COLD" and r["app_launch_time"] > 4000 :
        sf += (1 * lt)
    if r["app_launch_type"] == "HOT" and r["app_launch_time"] > 2000:
        sf += (1 * lt)   
        
    #conditions tolerance        
    if r["rage_count"] > 1 and r["rage_count"] < 4:
        st += ( r['rage_count'] * rt )
    if r["apiduration"] >= 1.5 and r["apiduration"] < 6:
        st += (r['apiduration'] * ad )
    if r["errorcount"] > 1 and r["errorcount"] < 4 : 
        st += 1
    if r['user_defiened_event_tolerance'] != 0 :
        st += (1 * eve )
    if r["app_launch_type"] == "COLD" and r["app_launch_time"] >= 1000 and r["app_launch_time"] < 4000:
        st += (1 * lt)
    if r["app_launch_type"] == "HOT" and r["app_launch_time"] >= 500 and r["app_launch_time"] < 2000:
        st += (1 * lt)
    
    #success
    if r['rage_count'] == 0 :
        summ += 1 
    if r['apiduration'] < 1.5:
        summ += 1
    if r['apistatus'] == 's':
        summ += 1
    if r['crs'] == 0:
        summ += 1
    if r['errorcount'] == 0 : 
        summ += 1
    if r['user_defiened_event_tolerance'] == 0 :
        summ += 1
    if r['user_defiened_event_frustrated'] == 0 :
        summ += 1
    if r["app_launch_type"] == "COLD" and r["app_launch_time"] < 1000 :
        summ += 1
    if r["app_launch_type"] == "HOT" and r["app_launch_time"] < 500:
        summ += 1  

    
    df.at[i,'s'] = summ
    df.at[i,'t'] = st
    df.at[i,'f'] = sf
df.head(10)

,usid,screen_duration,rage_count,apiduration,apistatus,crs,errorcount,app_launch_type,app_launch_time,user_defiened_event_tolerance,user_defiened_event_frustrated,case,s,t,f,Apdex,Star
0,1193007845,30400.0,0.0,1.303,s,0,0.0,COLD,949.0,20162019.0,0.0,Tolerance,7,2.5000,0.0000,0.85,3
1,1193007859,14000.0,0.0,0.047,s,0,0.0,COLD,668.0,0.0,0.0,sucess,8,0.0000,0.0000,1.00,4
2,1193007908,44400.0,0.0,1.521,s,0,0.0,COLD,1023.0,0.0,0.0,Tolerance,6,5.2815,0.0000,0.74,3
3,1193007933,59200.0,0.0,1.698,s,0,0.0,COLD,34913.0,0.0,0.0,fustrated,6,2.5470,3.0000,0.59,2
4,1193007956,25600.0,0.0,0.025,s,0,0.0,COLD,292.0,20337338.0,0.0,Tolerance,7,2.5000,0.0000,0.85,3
5,1193007959,7200.0,0.0,0.056,s,0,0.0,COLD,1652.0,0.0,0.0,Tolerance,7,3.0000,0.0000,0.83,3
6,1193008022,23200.0,0.0,0.056,s,0,0.0,COLD,827.0,0.0,0.0,sucess,8,0.0000,0.0000,1.00,4
7,1193008180,33400.0,0.0,8.571,s,0,0.0,COLD,493.0,0.0,0.0,fustrated,7,0.0000,12.8565,0.32,1
8,1193008197,58600.0,0.0,2.604,s,0,0.0,COLD,458.0,0.0,0.0,Tolerance,7,3.9060,0.0000,0.80,3
9,1193008217,60000.0,0.0,3.289,s,0,0.0,COLD,564.0,0.0,0.0,Tolerance,7,4.9335,0.0000,0.77,3


In [32]:
# apdex
def apdex(row):
    apdex = (row['s'] + (row['t']/2))/(row['s'] + row['t'] + row['f'])
    return round(apdex,2)

# Add the column 
df['Apdex'] = df.apply(apdex, axis=1)

df.head(10)

,usid,screen_duration,rage_count,apiduration,apistatus,crs,errorcount,app_launch_type,app_launch_time,user_defiened_event_tolerance,user_defiened_event_frustrated,case,s,t,f,Apdex,Star
0,1193007845,30400.0,0.0,1.303,s,0,0.0,COLD,949.0,20162019.0,0.0,Tolerance,7,2.5000,0.0000,0.87,3
1,1193007859,14000.0,0.0,0.047,s,0,0.0,COLD,668.0,0.0,0.0,sucess,8,0.0000,0.0000,1.00,4
2,1193007908,44400.0,0.0,1.521,s,0,0.0,COLD,1023.0,0.0,0.0,Tolerance,6,5.2815,0.0000,0.77,3
3,1193007933,59200.0,0.0,1.698,s,0,0.0,COLD,34913.0,0.0,0.0,fustrated,6,2.5470,3.0000,0.63,2
4,1193007956,25600.0,0.0,0.025,s,0,0.0,COLD,292.0,20337338.0,0.0,Tolerance,7,2.5000,0.0000,0.87,3
5,1193007959,7200.0,0.0,0.056,s,0,0.0,COLD,1652.0,0.0,0.0,Tolerance,7,3.0000,0.0000,0.85,3
6,1193008022,23200.0,0.0,0.056,s,0,0.0,COLD,827.0,0.0,0.0,sucess,8,0.0000,0.0000,1.00,4
7,1193008180,33400.0,0.0,8.571,s,0,0.0,COLD,493.0,0.0,0.0,fustrated,7,0.0000,12.8565,0.35,1
8,1193008197,58600.0,0.0,2.604,s,0,0.0,COLD,458.0,0.0,0.0,Tolerance,7,3.9060,0.0000,0.82,3
9,1193008217,60000.0,0.0,3.289,s,0,0.0,COLD,564.0,0.0,0.0,Tolerance,7,4.9335,0.0000,0.79,3


In [34]:
# Define the custom function
def star(row):
    star = row['Apdex']*4
    return round(star)

# Add the 'column3' column using the custom function
df['Star'] = df.apply(star, axis=1)

df[df['rage_count'] >= 4].head(10)


,usid,screen_duration,rage_count,apiduration,apistatus,crs,errorcount,app_launch_type,app_launch_time,user_defiened_event_tolerance,user_defiened_event_frustrated,case,s,t,f,Apdex,Star
285,1193013531,0.0,7.0,0.000,0,0,0.0,NaN,0.0,20337338.0,0.0,fustrated,4,2.5,14.0,0.26,1
296,1193013950,188800.0,4.0,0.767,s,0,0.0,COLD,820.0,20162019.0,0.0,Tolerance,6,2.5,8.0,0.44,2
822,1193026214,101800.0,5.0,0.929,s,0,0.0,HOT,9.0,0.0,0.0,fustrated,7,0.0,10.0,0.41,2
865,1193026427,42400.0,5.0,0.049,s,0,0.0,COLD,724.0,0.0,0.0,fustrated,7,0.0,10.0,0.41,2
1043,1193030208,291200.0,6.0,1.439,s,0,0.0,COLD,521.0,0.0,0.0,fustrated,7,0.0,12.0,0.37,1
1183,1193033317,129000.0,7.0,0.642,s,0,0.0,HOT,2.0,0.0,0.0,fustrated,7,0.0,14.0,0.33,1
1217,1193686224,186400.0,14.0,0.519,s,0,0.0,COLD,500.0,0.0,0.0,fustrated,7,0.0,28.0,0.20,1
1224,1193686517,85000.0,7.0,0.040,s,0,0.0,COLD,361.0,20320111.0,0.0,fustrated,6,2.5,14.0,0.32,1
1390,1193689338,38600.0,12.0,0.045,s,0,0.0,COLD,721.0,0.0,0.0,fustrated,7,0.0,24.0,0.23,1
1404,1193689611,36800.0,4.0,0.000,0,0,0.0,HOT,1.0,0.0,0.0,sucess,6,0.0,8.0,0.43,2


In [23]:
df.Star.value_counts()

4    4926
3    3024
1      99
2      94
0      22
Name: Star, dtype: int64

In [24]:
df.screen_duration.mean()

62867.573862939804

In [25]:
df.Star.mean()

3.559461114513166

In [12]:
df.to_csv('ux_score_apdex_test.csv')